In [1]:
import numpy as np
import pandas as pd
import requests, re
from bs4 import BeautifulSoup
from bs4 import Tag, NavigableString, Comment


# Create a function to parse through web pages

def parse_page(_d):
    data = filter(lambda x:x != '\n', [i for i in _d.find('table', {'id':'Browse'}).td.contents if isinstance(i, str) or i.name == 'a'])
    _next = next(data, None)
    while _next is not None:
        new_d, _n = {'title':_next.text, 'link':_next['href']}, next(data, None)
        if hasattr(_n, 'text'):
            yield new_d
            yield {'title':_n.text, 'link':_n['href'], 'about':next(data)}
        else:
            yield {**new_d, 'about':_n}   
        _next = next(data, None)  
        

# Identify the web page to start at and create a parsed text list with parse_page

d, r = BeautifulSoup(requests.get('https://www.mtsamples.com/site/pages/browse.asp?type=49-Orthopedic').text, 'html.parser'), []
r.append(list(parse_page(d)))
_c = [i for i in d.find('div', {'class':'Contrast'}).find_all('a') if i.text == '>']


# Use a while loop to continuously iterate over the links as long as the "next" (>) button is present. 
# At each iteration, scrape the next page and extract the headers with the link and description

while _c:
    d = BeautifulSoup(requests.get(f'https://www.mtsamples.com{_c[0]["href"]}').text, 'html.parser')
    r.append(list(parse_page(d)))
    _c = [i for i in d.find('div', {'class':'Contrast'}).find_all('a') if i.text == '>']
                          

In [2]:
# Specify medical specialty of web pages

orthopedic_web_pages = r
len(orthopedic_web_pages)

36

In [3]:
orthopedic_web_pages

[[{'title': 'AC Separation Revision & Hardware Removal',
   'link': '/site/pages/sample.asp?Type=49-Orthopedic&Sample=2697-AC Separation Revision & Hardware Removal',
   'about': 'Removal of the hardware and revision of right AC separation.  Loose hardware with superior translation of the clavicle implants.  Arthrex bioabsorbable tenodesis screws.'},
  {'title': 'Achilles Lengthening',
   'link': '/site/pages/sample.asp?Type=49-Orthopedic&Sample=1355-Achilles Lengthening',
   'about': 'Bilateral open Achilles lengthening with placement of short leg walking cast.'},
  {'title': 'Achilles Ruptured Tendon',
   'link': '/site/pages/sample.asp?Type=49-Orthopedic&Sample=64-Achilles Ruptured Tendon',
   'about': 'Right Achilles tendon rupture.'},
  {'title': 'Achilles Tendon Repair',
   'link': '/site/pages/sample.asp?Type=49-Orthopedic&Sample=1805-Achilles Tendon Repair',
   'about': 'Achilles tendon rupture, left lower extremity.  Primary repair left Achilles tendon.  The patient was steppi

In [4]:
# Create a url list for the medical records within the specified medical specialty 

orthopedic_url_list = []

for i in orthopedic_web_pages:
    for j in i:
        orthopedic_url_list.append('https://www.mtsamples.com' + j['link'])
        
len(orthopedic_url_list)

355

In [5]:
print(orthopedic_url_list)

['https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=2697-AC Separation Revision & Hardware Removal', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=1355-Achilles Lengthening', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=64-Achilles Ruptured Tendon', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=1805-Achilles Tendon Repair', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=932-Ankle Pain - Consult', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=933-Ankle Sprain - H&P', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=2115-Anterior Cervical Decompression', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=1830-Anterior Cervical Discectomy', 'https://www.mtsamples.com/site/pages/sample.asp?Type=49-Orthopedic&Sample=2106-Anterior Cervical Discectomy - 1', 'https://www.mtsampl

In [6]:
def get_medical_notes(url_link):

    global medical_notes
    medical_notes = []
    
    
    # We will loop through each of the url links provided as input for the function

    for i in url_link:

        url = i
        res = requests.get(url)
        res.raise_for_status()
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        
        
        # Here we can get page_title from <h1>

        title_el = soup.find('h1')
        page_title = title_el.text.strip()
        first_hr = title_el.find_next_sibling('hr')
        
        
        # Here we can get the page_title description after we see the text 'description'

        description_title = title_el.find_next_sibling('b', text=re.compile('description', flags=re.I))
        description_text_parts = []
        for s in description_title.next_siblings:
            if s is first_hr:
                break
            if isinstance(s, Tag):
                description_text_parts.append(s.text.strip())
            elif isinstance(s, NavigableString):
                description_text_parts.append(str(s).strip())
        description_text = '\n'.join(p for p in description_text_parts if p.strip())

        
        # Titles are all bold and uppercase
        
        titles = [b for b in first_hr.find_next_siblings('b') if b.text.strip().isupper()]
        
        
        # We can find the text between the titles and assign it to the title we see earlier

        docs = []
        
        for t in titles:
            text_parts = []
            for s in t.next_siblings:
                
                # go until next title
                
                if s in titles:
                    break
                if isinstance(s, Comment):
                    continue
                if isinstance(s, Tag):
                    if s.name == 'div':
                        break
                    text_parts.append(s.text.strip())
                elif isinstance(s, NavigableString):
                    text_parts.append(str(s).strip())
                    
            text = '\n'.join(p for p in text_parts if p.strip())
            
            docs.append({
                'title': t.text.strip(),
                'text': text
            })
            
            
        # Clean the text in docs such that we have one full_text document of medical notes
        
        lis = []

        for j in docs:
            for k in j.values():
                lis.append(k.rstrip('\n'))

        full_text = ''.join(lis)
        
        
        # Append to our medical_notes list

        medical_notes.append(full_text)

    len(medical_notes)

In [7]:
# Get and clean all medical notes for the specified medical specialty 

get_medical_notes(orthopedic_url_list)

orthopedic_medical_notes = medical_notes
len(orthopedic_medical_notes)

355

In [8]:
orthopedic_medical_notes

['PREOPERATIVE DIAGNOSIS:Right AC separation.POSTOPERATIVE DIAGNOSIS:Right AC separation.PROCEDURES:Removal of the hardware and revision of right AC separation.ANESTHESIA:General.BLOOD LOSS:100 cc.COMPLICATIONS:None.FINDINGS:Loose hardware with superior translation of the clavicle implants.IMPLANTS:Arthrex bioabsorbable tenodesis screws.SUMMARY:After informed consent was obtained and verified, the patient was brought to the operating room and placed supine on the operating table.  After uneventful general anesthesia was obtained, he was positioned in the beach chair and his right shoulder was sterilely prepped and draped in a normal fashion.  The incision was reopened and the hardware was removed without difficulty.  The AC joint was inspected and reduced.  An allograft was used to recreate the coracoacromial ligaments and then secured to decorticate with a bioabsorbable tenodesis screw and then to the clavicle.  And two separate areas that were split, one taken medially and one taken 

In [9]:
# Return medical notes and the labeled medical specialty as a dataframe

orthopedic_df = pd.DataFrame({'Medical Specialty': 'Orthopedics', 'Medical Notes': orthopedic_medical_notes})
print(len(orthopedic_df))
orthopedic_df.head()


355


,Medical Specialty,Medical Notes
0,Orthopedics,PREOPERATIVE DIAGNOSIS:Right AC separation.POS...
1,Orthopedics,PREOPERATIVE DIAGNOSIS:Idiopathic toe walker.P...
2,Orthopedics,CHIEF COMPLAINT:Achilles ruptured tendon.HISTO...
3,Orthopedics,PREOPERATIVE DIAGNOSIS:Achilles tendon rupture...
4,Orthopedics,IDENTIFICATION:The patient is a 15-year-old fe...


In [10]:
import pickle

# pickle dataframe as a checkpoint

pickle_out = open("orthopedic_df.pickle", "wb") # write file to bytes
pickle.dump(orthopedic_df, pickle_out)
pickle_out.close()